In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
load_dotenv()

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
with open("Response.json", "r") as file:
    RESPONSE_JSON = json.load(file)

In [ ]:
print(RESPONSE_JSON)

{'1': {'no': 1, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}, '2': {'no': 1, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}, '3': {'no': 1, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}}


In [ ]:
prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template="""
    Text: {text}
    You are an expert MCQ maker. Given the above text, it is you job to \
    create a quiz of {number} multiple choice questions (MCQs) with answers for {subject} students in {tone} tone.
    Make sure the questions are not repeated. check all the questions to be conforming text as well.
    Make sure to format your response in JSON format with the following structure: ad use it as a guide:
    Ensure to make {number} MCQs
    ### Response json:
    {RESPONSE_JSON}
    """
)

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Devstral-Small-2507",
    temperature=0.7,
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt, output_key='quiz', verbose=True)

In [ ]:
template = """
    Yor are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students, it is your job to \
    You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use max 50 words.
    if the quiz is not at per with the cognitive and analytical abilities of the students, \
    update the quiz questions which need to be changed and change the tone such that is perfectly fits the students abilities.
    Quiz_MCQs: {quiz}
    Check from an expert english Writer of the above quiz:
    """

In [ ]:
prompt2 = PromptTemplate(
    input_variables=["quiz", "subject"],
    template=template
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=prompt2, output_key='review', verbose=True)

In [ ]:
generate_evaluate_chain = SequentialChain(
    chains=[chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'RESPONSE_JSON'],
    output_variables=['quiz', 'review'],
    verbose=True)

In [ ]:
with open ("text.txt", "r") as file:
    text = file.read()

In [ ]:
print(text)

Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.

Generative AI has raised many ethical questions as it can be used for cyberc

In [ ]:
text
NUMBER = 5
SUBJECT = "AI"
TONE = "Formal"
RESPONSE_JSON = RESPONSE_JSON

In [ ]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text: Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, Anthropic, Meta AI, Micr


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Yor are an expert english grammarian and writer. Given a multiple choice quiz for AI students, it is your job to     You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use max 50 words.
    if the quiz is not at per with the cognitive and analytical abilities of the students,     update the quiz questions which need to be changed and change the tone such that is perfectly fits the students abilities.
    Quiz_MCQs:  {"4": {"no": 1, "question": "multiple choice question", "options": {"a": "option a", "b": "option b", "c": "option c", "d": "option d"}, "answer": "a"}}
    {"5": {"no": 1, "question": "multiple choice question", "options": {"a": "option a", "b": "option b", "c": "option c", "d": "option d"}, "answer": "a"}}

### Response json:
{
    "1": {
        "no": 1,
        "question": "What is generative AI primarily used for?",
        "options": {
 

In [ ]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Total Cost: ${cb.total_cost:.4f}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")

Total Tokens Used: 0
Total Cost: $0.0000
Prompt Tokens: 0
Completion Tokens: 0


In [ ]:
quiz=response.get('quiz')

In [ ]:
import re

# Remove markdown and whitespace
quiz_clean = re.sub(r'### Response json:|```', '', quiz).strip()

# Parse JSON
quiz_dict = json.loads(quiz_clean)

JSONDecodeError: Extra data: line 2 column 5 (char 157)

In [ ]:
quiz_dict

{'1': {'no': 1,
  'question': 'Which of the following is the first example of an algorithmically generated media?',
  'options': {'a': 'Large language models',
   'b': 'Markov chain',
   'c': 'Transformers',
   'd': 'AI planning systems'},
  'answer': 'b'},
 '2': {'no': 2,
  'question': 'Who created the computer program AARON to generate paintings?',
  'options': {'a': 'Andrey Markov',
   'b': 'Harold Cohen',
   'c': 'DeepMind',
   'd': 'OpenAI'},
  'answer': 'b'},
 '3': {'no': 3,
  'question': 'Which of the following was the first paper on Markov chains published by Andrey Markov?',
  'options': {'a': 'The first paper on Markov chains was published in 1906',
   'b': 'The first paper on Markov chains was published in 1913',
   'c': 'The first paper on Markov chains was published in 1921',
   'd': 'The first paper on Markov chains was published in 1929'},
  'answer': 'a'},
 '4': {'no': 4,
  'question': 'Which of the following is the first example of an algorithmically generated media?',

In [ ]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value['question']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['options'].items()
        ]
        )
    correct = value['answer']
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

[{'MCQ': 'Which of the following is the first example of an algorithmically generated media?',
  'Choices': 'a: Large language models | b: Markov chain | c: Transformers | d: AI planning systems',
  'Correct': 'b'},
 {'MCQ': 'Who created the computer program AARON to generate paintings?',
  'Choices': 'a: Andrey Markov | b: Harold Cohen | c: DeepMind | d: OpenAI',
  'Correct': 'b'},
 {'MCQ': 'Which of the following was the first paper on Markov chains published by Andrey Markov?',
  'Choices': 'a: The first paper on Markov chains was published in 1906 | b: The first paper on Markov chains was published in 1913 | c: The first paper on Markov chains was published in 1921 | d: The first paper on Markov chains was published in 1929',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is the first example of an algorithmically generated media?',
  'Choices': 'a: Large language models | b: Markov chain | c: Transformers | d: AI planning systems',
  'Correct': 'b'},
 {'MCQ': 'Which of the fo

In [ ]:
df = pd.DataFrame(quiz_table_data)

In [ ]:
df.head()

,MCQ,Choices,Correct
0,Which of the following is the first example of...,a: Large language models | b: Markov chain | c...,b
1,Who created the computer program AARON to gene...,a: Andrey Markov | b: Harold Cohen | c: DeepMi...,b
2,Which of the following was the first paper on ...,a: The first paper on Markov chains was publis...,a
3,Which of the following is the first example of...,a: Large language models | b: Markov chain | c...,b
4,Which of the following is the first example of...,a: Large language models | b: Markov chain | c...,b


In [ ]:
# df.to_csv("generated_quiz.csv", index=False)